In [1]:
import LevelsRelation_Extraction
%load_ext autoreload
%autoreload 2
import matplotlib
import cPickle, os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from snorkel.snorkel.snorkel import *
from snorkel.snorkel import *
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (18,6)
filename = "articles/"

In [2]:
Relations = LevelsRelation_Extraction.generateRelations(_filename= filename)
#create complete tuple object of all relation groups
AllRelations = LevelsRelation_Extraction.generateAllTupleRelations(_filename=filename)

/home/varun/Documents/Marker-Reader
dir: articles/nihms604917.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/zlj4536.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/PNAS-1998-Belinsky-11891-6.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/nihms389631.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/cam40003-1136.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/6246.full.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/nihms356718.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/5882.full.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/nihms551731.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/nihms143901.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/pone.0106748.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/nihms-489269.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/JO2012-709049.pdf.txt
/home/varun/Documents/Marker-Reader
dir: articles/1-s2.0-S0003267010012109-ma

Exception Exception: Exception('Candidate must have word_start and word_end attributes.',) in <generator object apply at 0x7fc7d121b960> ignored
Exception Exception: Exception('Candidate must have word_start and word_end attributes.',) in <generator object apply at 0x7fc7d121bf00> ignored


IndexError: list index out of range

In [ ]:
feats = None
relationsFile = filename + "Relations.txt"
try:
    with open(relationsFile, 'rb') as f:
        feats = cPickle.load(f)
except:
    %time Relations.extract_features()
    with open(relationsFile, 'w+') as f:
        cPickle.dump(Relations.feats, f)

DDL = DDLiteModel(Relations, feats)
print "Extracted {} features for each of {} mentions".format(DDL.num_feats(), DDL.num_candidates())

In [ ]:
"""Wrong ground truth files
with open('uids.pkl', 'rb') as f:DDL.open_mindtagger(num_sample=100, width='100%', height=1200)
    uids = cPickle.load(f)
with open('gt.pkl', 'rb') as f:
    gt = cPickle.load(f)

with open('examples/gene_tag_example/gt/uids.pkl', 'rb') as f:
    uids2 = cPickle.load(f)
with open('examples/gene_tag_example/gt/gt.pkl', 'rb') as f:
    gt2 = cPickle.load(f)
"""
DDL.update_gt(gt = gt[:50], uids = uids[:50])
DDL.set_holdout(validation_frac=0.5)

In [ ]:
DDL.open_mindtagger(num_sample=100, width='100%', height=1200)

In [ ]:
DDL.add_mindtagger_tags()
DDL.update_gt(gt=gt[50:], uids=uids[50:])

In [ ]:
keyWords_Measurement_Type = ["median", "mean", "standard deviation", "range", "level", "value", "change", "difference", "number", "concentration", "normal", "mode", "variance", "variation", "delta", "p=", "volume", "size", "mass", "weight", "quantity", "amount", "measurement", "measure", "abundance", "deviation", "magnitude", "score", "total", "degree", "amplitude", "extent", "mark", "proportion", "average", "increase", "decrease", "%", "percent", "portion", "depression", "elevation", "growth", "rise"]
keyWords_Test_Set = ["cohort", "normal", "subject", "patient", "participant", "group", "cell", "target", "baseline", "test", "trial", "experiment", "testcase", "testcase", "control", "batch", "category", "sample", "candidate", "case", "people", "member", "individual", "human", "population", "set", "person", "subset", "division", "sample set"]
negationWords = ["not", "nor", "neither"]


RelationsTuple = AllRelations[RelationsCounter]
#REMINDERS:
#   *when there are multiple attributes in the same sentence, the labeling functions will not
#   work, Example: Finally, for the three sub- jects with NE-CRPC, the age, serum PSA, and 
#   duration of ADT were 70.7 (68.1–79.2) years, PSA 0.4 (0.1–0.7) ng/ml, and 33.5 (5.2–174.2) months, respectively.
#   *Make a units matcher and input into the relations object 
# 1- distance far
def LF_distance_far_marker_to_level(m):
    # print m.lemmas
    # print m.dep_labels
    distance = abs(RelationsTuple[0].e2_idxs[0] - RelationsTuple[0].e1_idxs[0])
    if distance < 10:
        return 0
    else:
        return -1
        
# 2- distance far
def LF_distance_far_marker_to_measurement(m):
    distance = abs(RelationsTuple[1].e2_idxs[0] - RelationsTuple[1].e1_idxs[0])
    if distance < 10:
        return 0
    else:
        return -1
            
            
# 3- distance close
def LF_distance_close_marker_to_level(m):
    distance = abs(RelationsTuple[0].e2_idxs[0] - RelationsTuple[0].e1_idxs[0])
    if distance < 5:
        return 1
    else:
        return 0
        
# 4- distance close
def LF_distance_close_marker_to_measurement(m):
    distance = abs(RelationsTuple[2].e2_idxs[0] - RelationsTuple[2].e1_idxs[0])
    if distance < 5:
        return 1
    else:
        return 0
            
            
    
# 5- units 
def LF_units(m):
    found = False
    for unit in all_prefixes_units:
        if unit[0] in RelationsTuple[0].post_window2('lemmas',4) or unit[1] in RelationsTuple[0].post_window2('lemmas',4):
            found = True
    if found:
        return 1
    else:
        return 0
            
            
# 6- distance far
def LF_distance_far_data_set_to_measurement(m):
    distance = abs(RelationsTuple[2].e2_idxs[0] - RelationsTuple[2].e1_idxs[0])
    if distance < 9:
        return 0
    else:
        return -1 
            
# 7- measurement_type + to be + level: "the median was 50 mL"
def LF_typeIsALevel(m):
    return 1 if ('be' in RelationsTuple[1].post_window2('lemmas', 1) and 'be' in RelationsTuple[0].post_window2('lemmas',1) and abs(m.e3_idxs[0] - m.e2_idxs[0]) < 2) else 0 
    
# 8- Info + "of" + level: "Consistent with other reports, we observed a median (range) of 25 (6–170) ng/ml AGR2 in normal male subjects (n 1⁄4 18)"
def LF_infoOfLevel(m):
    return 1 if ('of' in RelationsTuple[2].post_window2('lemmas', 4) and 'of' in RelationsTuple[2].pre_window2('lemmas',4)) else 0 
        
# 9- If not words are present    
    # def presenceOfNot(m):
    #    for word in negationWords:
    #        if (word in m.post_window1('lemmas', 5)) and (word in m.pre_window2('lemmas', 5)):
    #            return True
    #    return False
    
# 10- If the measurement type contains common keywords
def LF_keyword_Measurement_Type(m):
    for word in keyWords_Measurement_Type:
        if (word in RelationsTuple[1].mention2(attribute='lemmas')):
            #if presenceOfNot(m):
            #    return -1
            #else:
            #    return 1
            return 1
    return 0
        
        
# 11- If the test set contains common keywords
def LF_keyword_Test_Set(m):
    for word in keyWords_Test_Set:
        if (word in RelationsTuple[2].mention2(attribute='lemmas')):
            #if presenceOfNot(m):
            #    return -1
            #else:
            #    return 1
            return 1
    return 0
    
# 12- If the test data has the a (n = #) notation after it
def LF_n_equals_notation(m):
    post_text = RelationsTuple[2].post_window2('text',20)
    normalized_post_text = unicodedata.normalize('NFKD', post_text).encode('ascii', 'ignore')
    regex_results = re.search('\(\s?n\s?\W\s?[\d\.]+\s?\)', normalized_post_text)
    if not regex_results == None:
        return 1
    else:
        return 0
    # Regex Expression for lemmas:       -LRB-\s?n\s?\W\s?[\d\.]+\s?-RRB-
    # Regex Expression for text:         \(\s?n\s?\W\s?[\d\.]+\s?\)
        
# 13- distance close test set to levels
def LF_distance_close_marker_to_test_set(m):
    distance = abs(RelationsTuple[1].e2_idxs[0] - RelationsTuple[1].e1_idxs[0])
    if distance < 7:
        return 1
    else:
        return 0
        
# 14- If there is a "in" between the test set and levels    "levels in normal patients"
def LF_of_L_MT(m):
    return 1 if "in" in RelationsTuple[0].post_window2('lemmas',6) and "in" in RelationsTuple[1].pre_window2('lemmas',6) else 0
    
#15 - If the units come right after the levels: 50 ng
def LF_units_place(m):
    distance = abs(RelationsTuple[3].e2_idxs[0] - RelationsTuple[0].e2_idxs[0])
    relationsCounter = relationsCounter + 1
    return 1 if distance < 3 else -1

In [ ]:
LFs = [LF_distance_far_marker_to_level, LF_distance_far_marker_to_measurement, LF_distance_close_marker_to_level, LF_distance_close_marker_to_measurement, LF_units, LF_distance_far_data_set_to_measurement, LF_units_place]
relationsCounter = 0
DDL.apply_lfs(LFs, clear=False)

In [ ]:
DDL.lowest_empirical_accuracy_lfs(n=10)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (12,4)
mu_seq = np.ravel([1e-9, 1e-5, 1e-3, 1e-1])
lf_opts = {'sample': False, 'verbose': True}
model_opts = {'sample': False, 'n_iter': 3000, 'alpha': 0.5, 'mu': mu_seq, 'bias': False, 'verbose': True}
%time DDL.train_model(method='lr', lf_opts=lf_opts, model_opts=model_opts)

In [ ]:
DDL.plot_calibration()